In [15]:
from torchtext import data
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import torch




In [16]:
TAGS = data.Field(unk_token = None)
TAGS.build_vocab(['0','S', 'C','G','T','B','D', 'A'])

In [18]:
path= '/Users/tancredicp/Desktop/sailing_app/data/ML Training Resources/'
all_files = glob.glob(path + "/*.csv")


Data = []
for filename in all_files:
    temporary_dict = {}

    dfSpecific = pd.read_csv(filename, index_col=None, header=0)
    dfSpecific.fillna(0, inplace= True)
    
    dfSpecificY = dfSpecific['tag']
    temporary_dict['tags'] = TAGS.numericalize((np.array(dfSpecificY)))
    

    dfSpecificX = dfSpecific.drop(columns=['tag', 'tag_name', 'boat_class', 'time'])
    
    scaler = StandardScaler()
    columnsBeingStandardize = ['lat', 'lon', 'sog', 'heel', 'clew_load','pitch']
    dfSpecificX[columnsBeingStandardize] = scaler.fit_transform(dfSpecificX[columnsBeingStandardize])
    
    
    array1 = np.array(dfSpecificX[['lat', 'lon', 'sog', 'cog']])
    
    temp = torch.from_numpy(array1)
    
    temporary_dict['track'] = temp
    
    Data.append(temporary_dict)
    del(temporary_dict)
    break


In [106]:
from torch.utils.data import Dataset
class SailingData(Dataset):
    def __init__ (self, pl):
        self.samples = []
        
        for d in pl:
            for track, tag in zip(d['track'], d['tags'][0]):
                self.samples.append((track, tag))
            
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
            return(self.samples[idx])

In [161]:
BATCH_SIZE = 1#the automatic batch size is 1
DatasetSailing= SailingData(Data)
train_iterator = torch.utils.data.DataLoader(DatasetSailing, batch_size = BATCH_SIZE)

In [162]:
DatasetSailing[1]

(tensor([ 0.5262, -0.5372,  1.5119, -0.3111], dtype=torch.float64), tensor(7))

In [163]:
len(TAGS.vocab)

9

In [179]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 120)
        self.fc2 = nn.Linear(120, 120)
        self.fc3 = nn.Linear(120,9)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)
        return x


net = Net()
net = net.double()

In [180]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [182]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_iterator, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
    print("Epoch: ", epoch, running_loss)
  
print('Finished Training')

Epoch:  0 5774.067815193655
Epoch:  1 5437.0967579334865
Epoch:  2 5435.048935347344
Epoch:  3 5432.8166192603085
Epoch:  4 5430.3483833338205
Epoch:  5 5427.663744591486
Epoch:  6 5424.366172811739
Epoch:  7 5420.3028747985245
Epoch:  8 5416.049299385432
Epoch:  9 5412.800676498631
Finished Training


In [219]:
lst = []
for i, data in enumerate(train_iterator, 0):
    inputs, labels = data
    o = net(inputs)
    lst.append(o.argmax().item())

In [221]:
print(lst)

[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 7, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 7, 7, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 

In [209]:
inti.argmax()

tensor(4, grad_fn=<NotImplemented>)

In [173]:
TAGS.tokenize('2')

['2']

In [178]:
TAGS

In [205]:
TAGS.tokenize(torch.t[0,1])

TypeError: split() missing 1 required positional argument: 'split_size'

In [196]:
TAGS.numericalize(['A', 'S'])

tensor([[2, 7]])